## Process for fuzzy matching company names
* This workflow is a first pass of drafting a mapping file of company names between 2 different data sources.

**What makes this process particularly challenging?**
* Company names across different sources may use different naming conventions and/or abbreviations
* User inputted company names may contain typos that result in variations for the same company

**Data Sources:**
For this demo, we are going to use an [SEC provided list](https://www.sec.gov/rules/other/4-460list.htm) of companies and try to find matches in a sample of manually entered companies


In [1]:
import jellyfish
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import re

* **Get SEC data fom sec.gov**
* **Create list of user inputted names**

In [2]:
sec_names = pd.read_html('https://www.sec.gov/rules/other/4-460list.htm')[0].iloc[1:,[1]]

companies = ['JP Morgan Chase', 'JPM', 'JP Morgan', 'Barnes & Noble, Inc.',
             'B&N', 'Barnes and Noble, Incorporation', 'Barnes and Nobles',
            'The Black & Decker Corporation', 'Black and Decker Corp', 'Delta Air Lines Inc.',
             'Delta Airlines', 'Delta Airlines Incorporation', 'Delta', 'H&R Block',
             'Marriott International Inc.', 'Marriott Intl.', 'Marriott International Incorporation',
             '3M', 'abercombie and fitch']

user_names = pd.DataFrame(companies) 

**Check what Jellyfish Metaphone looks like**

In [3]:
for company in companies[0:1]:
    print(jellyfish.metaphone(company))

JP MRKN XS


**Create empty dataframes for SEC, User Inputted, First Matching, Second Matching**

In [4]:
sec_df = pd.DataFrame(columns=['SEC Name', 'Acronym', 'Phonetic Representation'])
user_df = pd.DataFrame(columns=['Original User Name','User Inputed Name', 'Acronym', 'Phonetic Representation'])
matches_df = pd.DataFrame(columns=['Phonetic Representation', 'Phonetic Match Score'])
matches_df_two = pd.DataFrame(columns=['Acronym', 'Acronym Match Score'])

**Create list of the stopwords from NLTK and add our own**

In [5]:
stop = stopwords.words('english') + list(string.punctuation)

#we can't just remove common words in our list of company names because some words like 'American' repeat many times
new_words= ('inc','lp', 'llc', 'plc', 'limited', 'corporation', 'corp' ,'ltd', 'co',
            'Corporation', 'Inc', 'L.P.', 'LP', 'INC', 'LLC', 'Limited', 'Company',
            'Ltd', 'BV', 'bv', 'I', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'X', 'L.P.',
            'Lp', 'Co', 'CO', 'PARTNERS', 'Partners', 'Incorporated', 'incorporated', 'Corp',
            'Co.', "'", 'And', 'and', '&', 'The', 'the', 'Incorporation'
          )

for i in new_words:
    stop.append(i)

**Clean SEC Names and User Inputted Names**
   * Remove stopwords
   * Create Abbreviation, if length of Acronym is greater than 1
   * Create phonetic pronunciation from name

In [6]:
for row in sec_names.index:
    company = (sec_names[1][row])
    company = [i for i in word_tokenize(company) if i not in stop]
    company = " ".join(company)
    #print(company)
    company_abr = (''.join([c for c in company if c.isupper()]))
    if len(company_abr) > 1:
        company_abr = company_abr
    else:
        company_abr = ''
    company_phonetic = (jellyfish.metaphone(company))
    sec_df.loc[len(sec_df)]=[company,company_abr,company_phonetic] 

In [7]:
for row in user_names.index:
    company = (user_names[0][row])
    company = [i for i in word_tokenize(company) if i not in stop]
    company = " ".join(company)
    company_abr = (''.join([c for c in company if c.isupper()]))
    if len(company_abr) > 1:
        company_abr = company_abr
    else:
        company_abr = ''
    company_phonetic = (jellyfish.metaphone(company))
    originalname = (user_names[0][row])
    user_df.loc[len(user_df)]=[originalname, company,company_abr,company_phonetic] 

<b>Review at SEC Names</b>

In [8]:
sec_df.head()

,SEC Name,Acronym,Phonetic Representation
0,3Com,,KM
1,3M,,M
2,A.G. Edwards,AGE,AK ETWRTS
3,Abbott Laboratories,AL,ABT LBRTRS
4,Abercrombie Fitch,AF,ABRKRMB FX


<b>Review at User Inputted Names</b>

In [9]:
user_df.head()

,Original User Name,User Inputed Name,Acronym,Phonetic Representation
0,JP Morgan Chase,JP Morgan Chase,JPMC,JP MRKN XS
1,JPM,JPM,JPM,JPM
2,JP Morgan,JP Morgan,JPM,JP MRKN
3,"Barnes & Noble, Inc.",Barnes Noble,BN,BRNS NBL
4,B&N,B N,BN,B N


<b>Find nearest match to phonetic representation of User Inputted Name to SEC Names List</b>

In [10]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    s = df_2[key2].tolist()
    m_series = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))  
    for list_ in m_series:
        phonetic_match = [str((listItem[0])) for listItem in list_]
        score = [((listItem[1])) for listItem in list_]
        matches_df.loc[len(matches_df)]=[phonetic_match,score] 
    df_1 = pd.merge(df_1, matches_df, left_index=True, right_index=True)
    df_1['Phonetic Match Score'] = df_1['Phonetic Match Score'].str[0]
    df_1['Phonetic Representation_y'] = df_1['Phonetic Representation_y'].str[0]
    return df_1

fuzzymerge = fuzzy_merge(user_df, sec_df, 'Phonetic Representation', 'Phonetic Representation', threshold=92)

**Join Phonetic Match from fuzzy merge and see what the results look like**

In [11]:
join = pd.merge(fuzzymerge, sec_df, left_on=  ['Phonetic Representation_y'],
                   right_on= ['Phonetic Representation'], 
                   how = 'left')

join = join.drop(columns=['Acronym_y', 'Phonetic Representation'])
join = join.rename(columns = {'SEC Name':'Phonetic SEC Name'})
display(join)

,Original User Name,User Inputed Name,Acronym_x,Phonetic Representation_x,Phonetic Representation_y,Phonetic Match Score,Phonetic SEC Name
0,JP Morgan Chase,JP Morgan Chase,JPMC,JP MRKN XS,JP MRKN XS,100,J.P. Morgan Chase
1,JPM,JPM,JPM,JPM,M,90,3M
2,JP Morgan,JP Morgan,JPM,JP MRKN,JP MRKN XS,95,J.P. Morgan Chase
3,"Barnes & Noble, Inc.",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble
4,B&N,B N,BN,B N,BNK N YRK,86,Bank New York
5,"Barnes and Noble, Incorporation",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble
6,Barnes and Nobles,Barnes Nobles,BN,BRNS NBLS,BRNS NBL,94,Barnes Noble
7,The Black & Decker Corporation,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker
8,Black and Decker Corp,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker
9,Delta Air Lines Inc.,Delta Air Lines,DAL,TLT AR LNS,TLT AR LNS,100,Delta Air Lines


<b>Find nearest match on Acronym representation of User Inputted Name to SEC Names List</b>

In [12]:
def fuzzy_merge_two(df_1, df_2, key1, key2, threshold=90, limit=1):
    s = df_2[key2].tolist()
    m_series = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))  
    for list_ in m_series:
        phonetic_match = [str((listItem[0])) for listItem in list_]
        score = [((listItem[1])) for listItem in list_]
        matches_df_two.loc[len(matches_df_two)]=[phonetic_match,score] 
    df_1 = pd.merge(df_1, matches_df_two, left_index=True, right_index=True)
    df_1['Acronym Match Score'] = df_1['Acronym Match Score'].str[0]
    df_1['Acronym'] = df_1['Acronym'].str[0]
    return df_1

fuzzymergeTwo = fuzzy_merge_two(join, sec_df, 'Acronym_x', 'Acronym', threshold=92)
fuzzymergeTwo = fuzzymergeTwo[['Acronym','Acronym Match Score']]
fuzzymergeTwo.columns = ['SEC Acronym', 'Acronym Match Score']

<b>Join acronym matches if acronym exists</b>
   * Return a list of the SEC Names for the matching Acronym because there is a one to many relationship for Acronyms to SEC Names\
     * Each SEC Name only has 1 Acronym, but 1 Acronym can have many SEC Names

In [13]:
finaldf = join.join(fuzzymergeTwo, lsuffix='_left')
acronym_map = pd.DataFrame(columns=['Acronym', 'Company Names'])
    
for i, row in finaldf.iterrows():
    ac = (row['SEC Acronym'])
    names = []
    if ac == '':
        names is None
    else:
        for index, row in sec_df.iterrows():
            if ac == (row['Acronym']):
                names.append((row['SEC Name']))
    names = ', '.join(names)
    acronym_map.loc[len(acronym_map)]=[ac,names]    

    joinFinal = pd.merge(finaldf, acronym_map, left_on=  ['SEC Acronym'],
                   right_on= ['Acronym'], 
                   how = 'left')

joinFinal = joinFinal.drop_duplicates()
joinFinal = joinFinal.reset_index(drop=True)

**Final Cleaning / Rules for Success vs. Exception and display output**
   * Note that the final SEC name returned is a cleaned version of the actual SEC Name. We can join back to our original SEC Names dataset to get the original name back, if necessary.

In [14]:
joinFinal
joinFinal = joinFinal.rename(columns={'Original User Name':'Original Company Name', 
                                      'User Inputed Name':'Clean Original Name',
                                      'Acronym_x':'Clean Original Name Acronym',
                                      'Phonetic Representation_x':'Original Phonetic Representation', 
                                      'Phonetic Representation_y':'SEC Phonetic Representation',
                                      'Acronym_x':'Clean Original Name Acronym',
                                      'Acronym':'SEC Acronym',
                                      'Company Names':'Acronym SEC Names'})
joinFinal

,Original Company Name,Clean Original Name,Clean Original Name Acronym,Original Phonetic Representation,SEC Phonetic Representation,Phonetic Match Score,Phonetic SEC Name,SEC Acronym,Acronym Match Score,SEC Acronym,Acronym SEC Names
0,JP Morgan Chase,JP Morgan Chase,JPMC,JP MRKN XS,JP MRKN XS,100,J.P. Morgan Chase,JPMC,100,JPMC,J.P. Morgan Chase
1,JPM,JPM,JPM,JPM,M,90,3M,JP,90,JP,Jefferson-Pilot
2,JP Morgan,JP Morgan,JPM,JP MRKN,JP MRKN XS,95,J.P. Morgan Chase,JP,90,JP,Jefferson-Pilot
3,"Barnes & Noble, Inc.",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble,BN,100,BN,Barnes Noble
4,B&N,B N,BN,B N,BNK N YRK,86,Bank New York,BN,100,BN,Barnes Noble
5,"Barnes and Noble, Incorporation",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble,BN,100,BN,Barnes Noble
6,Barnes and Nobles,Barnes Nobles,BN,BRNS NBLS,BRNS NBL,94,Barnes Noble,BN,100,BN,Barnes Noble
7,The Black & Decker Corporation,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker,BD,100,BD,"Becton Dickinson, Black Decker"
8,Black and Decker Corp,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker,BD,100,BD,"Becton Dickinson, Black Decker"
9,Delta Air Lines Inc.,Delta Air Lines,DAL,TLT AR LNS,TLT AR LNS,100,Delta Air Lines,DAL,100,DAL,Delta Air Lines


**If the Original Phonetic Representation is found in the SEC Phonetic Representation, add a flag to pass it as a success.**
   * We can do this because many names are shortened when users write them in

In [15]:
joinFinal["Phonetic Flag"] = ''

for i, row in joinFinal.iterrows():
    original_phonetic = (row['Original Phonetic Representation'])
    sec_phonetic = (row['SEC Phonetic Representation'])
    if original_phonetic in sec_phonetic:
        flag = 'X'
    else:
        flag = ''
    joinFinal.at[i,["Phonetic Flag"]] = flag                              

**Final rules for passing a fuzzy match as a success or failure**

The following create a successful fuzzy match:
   * If the User Inputted Name phonetic sound is found within the SEC Phonetic Sound
   * If the Phonetic Match score is above 94 and the Original Name Phonetic Sound has more than 1 letter
   * If the Acronym Match score is greater than 95

In [16]:
def label(row):
    if row['Phonetic Flag'] == 'X':
        return row['Phonetic SEC Name']
    elif row['Phonetic Match Score'] > 94 and len(row['Original Phonetic Representation']) > 1:
        return row['Phonetic SEC Name']
    elif row['Acronym Match Score'] > 95:
        return row['Acronym SEC Names']
    else:
        return ''
joinFinal['SEC Name'] = joinFinal.apply (lambda row: label(row), axis=1)


In [17]:
joinFinal

,Original Company Name,Clean Original Name,Clean Original Name Acronym,Original Phonetic Representation,SEC Phonetic Representation,Phonetic Match Score,Phonetic SEC Name,SEC Acronym,Acronym Match Score,SEC Acronym,Acronym SEC Names,Phonetic Flag,SEC Name
0,JP Morgan Chase,JP Morgan Chase,JPMC,JP MRKN XS,JP MRKN XS,100,J.P. Morgan Chase,JPMC,100,JPMC,J.P. Morgan Chase,X,J.P. Morgan Chase
1,JPM,JPM,JPM,JPM,M,90,3M,JP,90,JP,Jefferson-Pilot,,
2,JP Morgan,JP Morgan,JPM,JP MRKN,JP MRKN XS,95,J.P. Morgan Chase,JP,90,JP,Jefferson-Pilot,X,J.P. Morgan Chase
3,"Barnes & Noble, Inc.",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble,BN,100,BN,Barnes Noble,X,Barnes Noble
4,B&N,B N,BN,B N,BNK N YRK,86,Bank New York,BN,100,BN,Barnes Noble,,Barnes Noble
5,"Barnes and Noble, Incorporation",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble,BN,100,BN,Barnes Noble,X,Barnes Noble
6,Barnes and Nobles,Barnes Nobles,BN,BRNS NBLS,BRNS NBL,94,Barnes Noble,BN,100,BN,Barnes Noble,,Barnes Noble
7,The Black & Decker Corporation,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker,BD,100,BD,"Becton Dickinson, Black Decker",X,Black Decker
8,Black and Decker Corp,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker,BD,100,BD,"Becton Dickinson, Black Decker",X,Black Decker
9,Delta Air Lines Inc.,Delta Air Lines,DAL,TLT AR LNS,TLT AR LNS,100,Delta Air Lines,DAL,100,DAL,Delta Air Lines,X,Delta Air Lines


**If a match, send to matches dataframe. If no match, send to excetion list**

In [18]:
matches = joinFinal[joinFinal['SEC Name'] != '']
exceptionList = joinFinal[joinFinal['SEC Name'] == '']

In [19]:
exceptionList

,Original Company Name,Clean Original Name,Clean Original Name Acronym,Original Phonetic Representation,SEC Phonetic Representation,Phonetic Match Score,Phonetic SEC Name,SEC Acronym,Acronym Match Score,SEC Acronym,Acronym SEC Names,Phonetic Flag,SEC Name
1,JPM,JPM,JPM,JPM,M,90,3M,JP,90,JP,Jefferson-Pilot,,


In [20]:
matches

,Original Company Name,Clean Original Name,Clean Original Name Acronym,Original Phonetic Representation,SEC Phonetic Representation,Phonetic Match Score,Phonetic SEC Name,SEC Acronym,Acronym Match Score,SEC Acronym,Acronym SEC Names,Phonetic Flag,SEC Name
0,JP Morgan Chase,JP Morgan Chase,JPMC,JP MRKN XS,JP MRKN XS,100,J.P. Morgan Chase,JPMC,100,JPMC,J.P. Morgan Chase,X,J.P. Morgan Chase
2,JP Morgan,JP Morgan,JPM,JP MRKN,JP MRKN XS,95,J.P. Morgan Chase,JP,90,JP,Jefferson-Pilot,X,J.P. Morgan Chase
3,"Barnes & Noble, Inc.",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble,BN,100,BN,Barnes Noble,X,Barnes Noble
4,B&N,B N,BN,B N,BNK N YRK,86,Bank New York,BN,100,BN,Barnes Noble,,Barnes Noble
5,"Barnes and Noble, Incorporation",Barnes Noble,BN,BRNS NBL,BRNS NBL,100,Barnes Noble,BN,100,BN,Barnes Noble,X,Barnes Noble
6,Barnes and Nobles,Barnes Nobles,BN,BRNS NBLS,BRNS NBL,94,Barnes Noble,BN,100,BN,Barnes Noble,,Barnes Noble
7,The Black & Decker Corporation,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker,BD,100,BD,"Becton Dickinson, Black Decker",X,Black Decker
8,Black and Decker Corp,Black Decker,BD,BLK TKR,BLK TKR,100,Black Decker,BD,100,BD,"Becton Dickinson, Black Decker",X,Black Decker
9,Delta Air Lines Inc.,Delta Air Lines,DAL,TLT AR LNS,TLT AR LNS,100,Delta Air Lines,DAL,100,DAL,Delta Air Lines,X,Delta Air Lines
10,Delta Airlines,Delta Airlines,DA,TLT ARLNS,TLT AR LNS,95,Delta Air Lines,DAL,90,DAL,Delta Air Lines,,Delta Air Lines


**View the final matches with just Original Name and SEC Name (clean)**

In [21]:
matches_no_details = matches[['Original Company Name', 'SEC Name']]
matches_no_details

,Original Company Name,SEC Name
0,JP Morgan Chase,J.P. Morgan Chase
2,JP Morgan,J.P. Morgan Chase
3,"Barnes & Noble, Inc.",Barnes Noble
4,B&N,Barnes Noble
5,"Barnes and Noble, Incorporation",Barnes Noble
6,Barnes and Nobles,Barnes Noble
7,The Black & Decker Corporation,Black Decker
8,Black and Decker Corp,Black Decker
9,Delta Air Lines Inc.,Delta Air Lines
10,Delta Airlines,Delta Air Lines
